In [1]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

In [3]:
from transformers import pipeline
pipe = pipeline("text-classification",
                model="j-hartmann/emotion-english-distilroberta-base",
                device = 0,
                top_k=None
                )
pipe('i love this')


Device set to use cpu


[[{'label': 'joy', 'score': 0.9834631085395813},
  {'label': 'sadness', 'score': 0.00798109546303749},
  {'label': 'surprise', 'score': 0.003793600481003523},
  {'label': 'anger', 'score': 0.002090137917548418},
  {'label': 'neutral', 'score': 0.00157511280849576},
  {'label': 'fear', 'score': 0.0006469090585596859},
  {'label': 'disgust', 'score': 0.00045007967855781317}]]

In [4]:
books['description'][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [5]:
pipe(books['description'][0])

[[{'label': 'fear', 'score': 0.6548405885696411},
  {'label': 'neutral', 'score': 0.16985228657722473},
  {'label': 'sadness', 'score': 0.11640921980142593},
  {'label': 'surprise', 'score': 0.02070065587759018},
  {'label': 'disgust', 'score': 0.019100677222013474},
  {'label': 'joy', 'score': 0.01516144908964634},
  {'label': 'anger', 'score': 0.003935146611183882}]]

In [6]:
pipe(books['description'][0].split('.'))

[[{'label': 'surprise', 'score': 0.7296020984649658},
  {'label': 'neutral', 'score': 0.14038600027561188},
  {'label': 'fear', 'score': 0.06816228479146957},
  {'label': 'joy', 'score': 0.04794260859489441},
  {'label': 'anger', 'score': 0.009156366810202599},
  {'label': 'disgust', 'score': 0.0026284765917807817},
  {'label': 'sadness', 'score': 0.002122163539752364}],
 [{'label': 'neutral', 'score': 0.44937002658843994},
  {'label': 'disgust', 'score': 0.27359163761138916},
  {'label': 'joy', 'score': 0.10908330976963043},
  {'label': 'sadness', 'score': 0.09362746775150299},
  {'label': 'anger', 'score': 0.04047830402851105},
  {'label': 'surprise', 'score': 0.026970159262418747},
  {'label': 'fear', 'score': 0.006879047024995089}],
 [{'label': 'neutral', 'score': 0.6462159752845764},
  {'label': 'sadness', 'score': 0.24273329973220825},
  {'label': 'disgust', 'score': 0.04342271760106087},
  {'label': 'surprise', 'score': 0.028300564736127853},
  {'label': 'joy', 'score': 0.014211

In [10]:
sentence = books['description'][0].split('.')
predictions = pipe(sentence)

In [12]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296020984649658},
 {'label': 'neutral', 'score': 0.14038600027561188},
 {'label': 'fear', 'score': 0.06816228479146957},
 {'label': 'joy', 'score': 0.04794260859489441},
 {'label': 'anger', 'score': 0.009156366810202599},
 {'label': 'disgust', 'score': 0.0026284765917807817},
 {'label': 'sadness', 'score': 0.002122163539752364}]

In [13]:
sorted(predictions[0], key=lambda x: x['label'])

[{'label': 'anger', 'score': 0.009156366810202599},
 {'label': 'disgust', 'score': 0.0026284765917807817},
 {'label': 'fear', 'score': 0.06816228479146957},
 {'label': 'joy', 'score': 0.04794260859489441},
 {'label': 'neutral', 'score': 0.14038600027561188},
 {'label': 'sadness', 'score': 0.002122163539752364},
 {'label': 'surprise', 'score': 0.7296020984649658}]

In [14]:
import numpy as np

emotion_labels = ['anger','disgust','fear','joy','sadness','surprise','neutral']
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion(predictions):
    per_emotion_score = {label: [] for label in emotion_labels}

    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x['label'])
        for index, label in enumerate(emotion_labels):
            per_emotion_score[label].append(sorted_predictions[index]['score'])
    return {label: np.max(scores) for label, scores in per_emotion_score.items()}


In [15]:
from tqdm import tqdm
emotion_labels = ['anger','disgust','fear','joy','sadness','surprise','neutral']
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books['isbn13'][i])
    sentence = books['description'][i].split('.')
    predictions = pipe(sentence)
    max_scores = calculate_max_emotion(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [24:03<00:00,  3.60it/s]  


In [16]:
emotion_df = pd.DataFrame(emotion_scores)
emotion_df['isbn13'] = isbn


In [17]:
emotion_df.head()

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273592,0.928168,0.932798,0.646216,0.967158,0.729602,9780002005883
1,0.612620,0.348285,0.942528,0.704422,0.887940,0.111690,0.252546,9780002261982
2,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078766,9780006178736
3,0.351484,0.150722,0.360706,0.251882,0.732684,0.111690,0.078766,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475880,0.078766,9780006280934


In [18]:
books = pd.merge(books, emotion_df, on='isbn13')

In [19]:
books.to_csv('books_with_emotion.csv', index = False)